In [337]:
import pandas as pd
import numpy as np
import plotly.express as px
filename = "../datasets/house_prices.csv"

In [338]:
df = pd.read_csv(filename)
df.shape

(21616, 21)

In [339]:
df.dtypes

id               float64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view             float64
condition        float64
grade            float64
sqft_above       float64
sqft_basement    float64
yr_built         float64
yr_renovated     float64
zipcode          float64
lat              float64
long             float64
sqft_living15    float64
sqft_lot15       float64
dtype: object

In [340]:
# clearing duplicates
df = df.drop_duplicates()

In [341]:
# Validating features:

to_remove = pd.concat([df.loc[(df.bedrooms <=0) & (df.bathrooms <=0)], 
                      df.loc[(df.sqft_living <= 0) | (df.sqft_lot <= 0) | (df.sqft_above <= 0) | (df.sqft_basement < 0) 
                             | (df.sqft_living15 < 0) | (df.sqft_lot15 < 0) | (df.price < 0) | (df.price.isnull())]]).drop_duplicates()

for row_idx in to_remove.index:
    df.drop(row_idx, axis=0, inplace=True)

In [342]:
# parse the date column:

df['date'] = pd.to_datetime(df.date, errors='coerce')

In [343]:
# delete samples with no date

for row_idx in df.loc[df.date.isnull()].index:
    print(row_idx)
    df.drop(row_idx, axis=0, inplace=True)

10816


In [344]:
# replace date column with year month and day columns:
df['sale_year'] = df.date.dt.year
df['sale_month'] = df.date.dt.month
df['sale_day'] = df.date.dt.weekday

In [345]:
# Removing redundant columns:
df.drop(columns=['id','date','zipcode'], inplace=True)

In [346]:
# Create column for age := sale_year - max(yr_built, yr_renovated):

df['age'] = df.sale_year - np.maximum(df.yr_built, df.yr_renovated)

# there are 18 rows with negative age - need to deal with that?
df.loc[df.age < 0]

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,sale_year,sale_month,sale_day,age
1763,597326.0,4,4.00,3570,8250,2.0,0,0.0,3.0,10.0,...,2015.0,0.0,47.5784,-122.226,2230.0,10000.0,2014,6,2,-1.0
2295,585000.0,5,2.75,2350,4178,1.5,0,0.0,3.0,8.0,...,1922.0,2015.0,47.6388,-122.300,1920.0,4178.0,2014,7,0,-1.0
2687,385195.0,1,1.00,710,6000,1.5,0,0.0,3.0,6.0,...,2015.0,0.0,47.5756,-122.316,1440.0,4800.0,2014,10,2,-1.0
7097,285000.0,4,3.50,2770,10505,2.0,0,0.0,3.0,8.0,...,1940.0,2015.0,47.7412,-122.355,1760.0,10505.0,2014,10,1,-1.0
7526,614285.0,5,2.75,2730,6401,2.0,0,0.0,3.0,8.0,...,2015.0,0.0,47.7685,-122.160,2520.0,6126.0,2014,12,2,-1.0
8039,455000.0,2,1.50,1200,1259,2.0,0,0.0,3.0,8.0,...,2015.0,0.0,47.6001,-122.298,1320.0,1852.0,2014,6,1,-1.0
11599,850000.0,4,3.25,3090,6744,2.0,0,4.0,3.0,9.0,...,1923.0,2015.0,47.7680,-122.388,2020.0,6656.0,2014,5,3,-1.0
14489,500000.0,2,2.25,1570,1269,2.0,0,0.0,3.0,9.0,...,2015.0,0.0,47.6514,-122.385,1570.0,6000.0,2014,8,1,-1.0
14859,805000.0,5,2.50,4600,19831,1.0,0,3.0,3.0,9.0,...,1956.0,2015.0,47.7608,-122.378,2890.0,19831.0,2014,6,4,-1.0
15687,825000.0,3,2.50,3400,38400,1.0,0,4.0,3.0,8.0,...,1955.0,2015.0,47.7611,-122.372,3400.0,24338.0,2014,10,0,-1.0


In [347]:
# Create sqft ratio columns:

df['sqft_living_ratio'] = df.sqft_living15 / df.sqft_living
df['sqft_lot_ratio'] = df.sqft_lot15 / df.sqft_lot
# no division by zero because i removed rows with sqft_living==0 or sqft_lot==0

print(df[['sqft_living', 'sqft_living15','sqft_living_ratio']])
print("\n")
print(df[['sqft_lot', 'sqft_lot15','sqft_lot_ratio']])

       sqft_living  sqft_living15  sqft_living_ratio
0             1180         1340.0           1.135593
1             2570         1690.0           0.657588
2              770         2720.0           3.532468
3             1960         1360.0           0.693878
4             1680         1800.0           1.071429
...            ...            ...                ...
21611         1530         1530.0           1.000000
21612         2310         1830.0           0.792208
21613         1020         1020.0           1.000000
21614         1600         1410.0           0.881250
21615         1020         1020.0           1.000000

[21602 rows x 3 columns]


       sqft_lot  sqft_lot15  sqft_lot_ratio
0          5650      5650.0        1.000000
1          7242      7639.0        1.054819
2         10000      8062.0        0.806200
3          5000      5000.0        1.000000
4          8080      7503.0        0.928589
...         ...         ...             ...
21611      1131      1509.0 

In [349]:
px.scatter(x=df.head(100).sqft_living_ratio, y=df.head(100).price, trendline="ols",
          title=f"sqft_living_ratio vs price:\nPearson Correlation = {7}",
          labels=dict(x="sqft_living_ratio", y="price"))

information about the dataset:
    https://geodacenter.github.io/data-and-lab/KingCounty-HouseSales2015/



What to do with specific columns:
* id column:
    - turn it into index column?
    - delete it?

* date column:
    - convert to year, month, day columns?
    - use it to get the 'age' of the house?
    - delete it afterwards

* numerical values that don't have natural ordering - what to do with them?:
    - zipcode, lat, long

* price column - this is the response vector

Validations:
* remove duplicates

* Values that should be validated as non-negative:
    - bedrooms, bathrooms, floors
    - sqft_living, sqft_lot, sqft_above, sqft_basement, sqft_living15, sqft_lot15
    - price
    
* yr_built and yr_yr_renovated columns:
    - should validate that the year make sense.
    
* condition column:
    - should be integer between 1 and 5
    
* view column:
    - should be integer between 0 and 4

Custum Features:
* age
* ratio of sqft_living and sqft_living15
* ratio of sqft_lot and sqft_lot15
    
Questions:
* which values here are categorical and needs to be dealt with as such?
    - date, and i think i delt with it alright
* new features that can be created from existing features?

Usefull things:
* select rows with conditions on strings:
    https://kanoki.org/2019/03/27/pandas-select-rows-by-condition-and-string-operations/
* select rows with conditions on numbers:
    https://www.stackvidhya.com/select-rows-from-dataframe/#:~:text=Pandas%20Dataframe%20stores%20data%20in,column_name'%5D%20%3D%3D%20value%5D.
* convert strings to floats:
https://datatofish.com/convert-string-to-float-dataframe/